In [10]:
from glob import glob
from os.path import join
import pandas as pd
import numpy as np

In [2]:
common_path = '/Volumes/MMIS-Saraiv/Datasets/Miltiadous Dataset/features'
all_directories = glob(join(common_path, '*'))
all_directories = [directory for directory in all_directories if not directory.endswith('.csv')]

In [3]:
FEATURES_SELECTED = ['Spectral#RelativePower#C3#beta1', 'Spectral#EdgeFrequency#C3#beta3', 'Spectral#RelativePower#C3#gamma', 'Spectral#EdgeFrequency#C4#alpha1', 'Spectral#RelativePower#C4#beta3', 'Spectral#EdgeFrequency#C4#beta3', 'Spectral#EdgeFrequency#C4#gamma', 'Spectral#Flatness#Cz#theta', 'Spectral#PeakFrequency#Cz#theta', 'Spectral#EdgeFrequency#Cz#beta3', 'Spectral#EdgeFrequency#Cz#gamma', 'Spectral#PeakFrequency#Cz#gamma', 'Spectral#RelativePower#F3#beta1', 'Spectral#Diff#F4#delta', 'Spectral#RelativePower#F7#beta3', 'Spectral#EdgeFrequency#F7#beta3', 'Spectral#RelativePower#F7#gamma', 'Spectral#RelativePower#F8#beta1', 'Spectral#EdgeFrequency#F8#beta3', 'Spectral#RelativePower#Fp1#beta1', 'Spectral#EdgeFrequency#Fp1#beta3', 'Spectral#Diff#Fp2#delta', 'Spectral#RelativePower#Fp2#beta1', 'Spectral#RelativePower#Fp2#beta3', 'Spectral#Diff#Fpz#beta2', 'Spectral#Entropy#O1#delta', 'Spectral#RelativePower#O1#beta2', 'Spectral#EdgeFrequency#O1#beta2', 'Spectral#EdgeFrequency#O1#beta3', 'Spectral#RelativePower#O2#delta', 'Spectral#PeakFrequency#O2#alpha1', 'Spectral#RelativePower#O2#beta1', 'Spectral#RelativePower#O2#beta3', 'Spectral#Diff#P3#beta1', 'Spectral#RelativePower#P3#beta3', 'Spectral#RelativePower#Pz#alpha1', 'Spectral#EdgeFrequency#Pz#beta3', 'Spectral#RelativePower#T4#alpha1', 'Spectral#RelativePower#T4#beta3', 'Spectral#RelativePower#T4#gamma', 'Spectral#EdgeFrequency#T5#beta2', 'Hjorth#Complexity#T5', 'Hjorth#Complexity#P4', 'Hjorth#Complexity#F7', 'Hjorth#Complexity#T4', 'Hjorth#Complexity#F8', 'Hjorth#Complexity#T3', 'Hjorth#Mobility#P3', 'PLI#Frontal(L)-Temporal(R)#alpha1', 'PLI#Frontal(L)-Occipital(L)#alpha1', 'PLI#Frontal(R)-Temporal(R)#alpha1', 'PLI#Temporal(R)-Parietal(R)#alpha1', 'PLI#Temporal(R)-Occipital(L)#alpha1', 'PLI#Parietal(R)-Occipital(L)#alpha1', 'PLI#Occipital(L)-Occipital(R)#alpha1', 'PLI#Temporal(R)-Occipital(R)#alpha2', 'PLI#Parietal(R)-Occipital(L)#alpha2', 'COH#Frontal(L)-Frontal(R)#theta', 'COH#Frontal(L)-Occipital(L)#theta', 'COH#Frontal(L)-Occipital(R)#alpha1', 'COH#Frontal(R)-Occipital(L)#alpha1', 'COH#Parietal(R)-Occipital(L)#alpha1', 'COH#Frontal(L)-Frontal(R)#alpha2', 'COH#Frontal(L)-Occipital(R)#alpha2', 'COH#Parietal(R)-Occipital(L)#alpha2', 'COH#Parietal(R)-Occipital(R)#alpha2', 'COH#Occipital(L)-Occipital(R)#alpha2', 'COH#Frontal(L)-Occipital(L)#beta1', 'COH#Temporal(R)-Parietal(R)#beta1', 'COH#Parietal(R)-Occipital(R)#beta1', 'COH#Frontal(L)-Parietal(L)#beta2', 'COH#Frontal(R)-Occipital(L)#beta2', 'COH#Frontal(L)-Temporal(R)#beta3', 'COH#Frontal(L)-Parietal(L)#beta3', 'COH#Frontal(L)-Occipital(L)#beta3', 'COH#Frontal(L)-Occipital(R)#beta3', 'COH#Frontal(R)-Occipital(L)#beta3', 'COH#Temporal(L)-Occipital(R)#beta3', 'COH#Frontal(L)-Occipital(R)#gamma', 'COH#Frontal(R)-Occipital(R)#gamma']


In [59]:
from sklearn.cluster import KMeans, DBSCAN

for subject_path in all_directories:
    all_multiple_files = glob(join(subject_path, '*$Multiple.csv'))
    if len(all_multiple_files) != 0:  # underrepresented target
        print("##################")
        print("Subject: ", subject_path.split('/')[-1])
        # Read them all
        all_data = [pd.read_csv(file, index_col=0) for file in all_multiple_files]
        # Concatenate all their columns in a single DataFrame
        all_data = pd.concat(all_data, axis=1)
        # Keep only selected features
        FEATURES_SELECTED = [col for col in all_data.columns if col in FEATURES_SELECTED]
        selected_features = all_data[FEATURES_SELECTED]
        # drop nans
        selected_features = selected_features.dropna()
        
        # Normalise each row between 0 and 1
        # Normalise each feature (column) between 0 and 1
        selected_features = (selected_features - selected_features.min()) / (selected_features.max() - selected_features.min())

        # DISTORTION / AUGMENTATION
        # Apply noise to each row
        # Apply noise to each row
        """
        for row in range(len(selected_features)):
            data = selected_features.iloc[row].values
            data = data + np.random.normal(loc=0, scale=0.2, size=data.shape)
            selected_features.iloc[row] = data
        """
        
        # Normalise across all rows (examples)
        selected_features = (selected_features - selected_features.min()) / (selected_features.max() - selected_features.min())
        
        # STATISTICAL TESTS
        independent_rows = []
        dependent_rows = []
        from scipy.stats import mannwhitneyu
        for i in range(len(selected_features)):
            for j in range(i, len(selected_features)):
                # Perform <Mann-Whitney U test>
                u_stat, p = mannwhitneyu(selected_features.iloc[i], selected_features.iloc[j])
                # interpret
                # p > alpha: Likely come from the same distribution (fail to reject H0)
                # p <= alpha: Likely come from different distributions (reject H0)
                alpha = 0.05
                # print the p-values with 3 decimal places
                if p > alpha:
                    #print(f'Rows ({i}, {j}): Likely come from the same subject (fail to reject H0). u_stat={u_stat}, p={p:.3f}')
                    if i != j:
                        dependent_rows.append((i, j))
                else:
                    #print(f'Rows ({i}, {j}): Likely come from different subjects (reject H0) u_stat={u_stat}, p={p:.3f}')
                    independent_rows.append((i, j))
        
        print("Number of independent pairs:", len(independent_rows))
        
        #print("##################")
        import networkx as nx
        G = nx.Graph()
        # Add a node for each row
        for i in range(len(selected_features)):
            G.add_node(i)
        # Add an edge for each pair with p-value less than 0.05
        for i, j in independent_rows:
            G.add_edge(i, j)
        # Find the largest clique
        cliques = list(nx.find_cliques(G))
        #print("All cliques:")
        #for clique in cliques:
        #    print(clique)
        largest_clique = max(cliques, key=len)
        #print("The largest set of rows where all pairs have p<0.05 is:")
        #print(largest_clique)
        K = len(largest_clique)
        print(f'Length of the largest clique, {K}, will be K.')
        
        # Use a clustering algorithm
        K = len(largest_clique)
        kmeans = KMeans(n_clusters=K, random_state=0).fit(selected_features)
        labels = kmeans.labels_
        #print(f'Cluster labels: {labels}')
        
        # Check if any of the independent pairs are in the same cluster
        to_exclude = []
        for i, j in independent_rows:
            if labels[i] == labels[j]:
                #print(f'Rows ({i}, {j}) are independent but in the same cluster')
                to_exclude.append((i, j))
        
        
        print("Independent pairs that can be safely used because they're not clustered together:")
        safe_pairs = []
        for i, j in independent_rows:
            if (i, j) not in to_exclude and (j, i) not in to_exclude:
                safe_pairs.append((i, j))
                print(f'Pair ({i}, {j})')
                
        cliques_K = [clique for clique in cliques if len(clique) == K]
        print(f"Cliques with {K} elements with safe pairs to use:")
        for clique in cliques_K:
            if all((i, j) in safe_pairs or (j, i) in safe_pairs for i in clique for j in clique if i != j):
                print(clique)
        
        #break
    